In [ ]:
import subprocess
import sys
import os
import time

# ==============================================================================
# GIẢI PHÁP TRIỆT ĐỂ: PATCH OUTLINES ĐỂ BỎ QUA PYAIRPORTS
# ==============================================================================
print("🛠️ Đang cài đặt và patch thư viện...")

# Gỡ bỏ để tránh xung đột
subprocess.run("pip uninstall -y tensorflow keras vllm", shell=True, 
               stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

# Cài đặt các gói cần thiết
cmd = [
    sys.executable, "-m", "pip", "install", 
    "vllm",
    "xformers",
    "pyngrok",
    "numpy<2.0.0",
    "transformers[torch]",
    "pycountry",
    "outlines"
]
subprocess.run(cmd, check=True, stdout=subprocess.DEVNULL)

# Vá lỗi Cuda
if not os.path.exists("/usr/local/cuda/lib64/libcuda.so"):
    subprocess.run("ln -s /usr/local/cuda/lib64/stubs/libcuda.so /usr/local/cuda/lib64/libcuda.so", 
                   shell=True, stderr=subprocess.DEVNULL)

# ==============================================================================
# PATCH OUTLINES ĐỂ BỎ QUA PYAIRPORTS
# ==============================================================================
def patch_outlines():
    """Vá file outlines/types/__init__.py để không import airports"""
    try:
        import outlines
        outlines_path = os.path.dirname(outlines.__file__)
        types_init = os.path.join(outlines_path, "types", "__init__.py")
        
        if os.path.exists(types_init):
            with open(types_init, 'r') as f:
                content = f.read()
            
            # Thay đổi: from . import airports, countries -> from . import countries
            if 'airports' in content:
                new_content = content.replace(
                    'from . import airports, countries',
                    'from . import countries  # airports removed - pyairports not available'
                )
                
                with open(types_init, 'w') as f:
                    f.write(new_content)
                
                print("✅ Đã patch outlines/types/__init__.py")
                return True
        
        print("⚠️ Không tìm thấy file cần patch")
        return False
    except Exception as e:
        print(f"⚠️ Không thể patch: {e}")
        return False

# Thực hiện patch
patched = patch_outlines()

# ==============================================================================
# GIẢI PHÁP DỰ PHÒNG: TẠO MODULE GIẢ
# ==============================================================================
if not patched:
    print("📦 Tạo module pyairports giả...")
    import site
    site_packages = site.getsitepackages()[0]
    pyairports_dir = os.path.join(site_packages, "pyairports")
    os.makedirs(pyairports_dir, exist_ok=True)
    
    # Tạo __init__.py
    with open(os.path.join(pyairports_dir, "__init__.py"), "w") as f:
        f.write("# Dummy module\n")
    
    # Tạo airports.py với AIRPORT_LIST rỗng
    with open(os.path.join(pyairports_dir, "airports.py"), "w") as f:
        f.write("AIRPORT_LIST = []  # Dummy data\n")
    
    print("✅ Đã tạo pyairports giả")

print("✅ Hoàn tất cài đặt!")

# ==============================================================================
# KHỞI CHẠY SERVER
# ==============================================================================
from pyngrok import ngrok

# --- CẤU HÌNH ---
# Model ID hoặc đường dẫn local
# Chọn 1 trong các model sau tùy VRAM của bạn:

# 🔹 Model nhỏ (~8GB VRAM) - Nhanh, phù hợp GPU thường
# MODEL_ID = "Qwen/Qwen2.5-7B-Instruct-AWQ"

# 🔹 Model vừa (~16GB VRAM) - Cân bằng
# MODEL_ID = "Qwen/Qwen2.5-14B-Instruct-AWQ"

# 🔹 Model lớn (~18-20GB VRAM với AWQ) - Phù hợp T4x2
MODEL_ID = "Qwen/Qwen2.5-32B-Instruct-AWQ"

# 🔹 Hoặc dùng đường dẫn local (Kaggle dataset)
# MODEL_ID = "/kaggle/input/qwen-3/transformers/32b/1"

# Tên model hiển thị trong API
SERVED_MODEL_NAME = "Qwen2.5-32B-Instruct-AWQ"

NGROK_AUTH_TOKEN = "2loaj9mCBrswSefBAn7cebLdfDT_6o8gA9vCBpVDi7P3m2c2e"

# Thiết lập Ngrok
try:
    ngrok.set_auth_token(NGROK_AUTH_TOKEN)
    ngrok.kill()
    public_url = ngrok.connect(8000).public_url
except Exception as e:
    print(f"❌ Lỗi Ngrok: {e}")
    sys.exit(1)

print("\n" + "="*60)
print(f"🚀 SERVER INFO DASHBOARD")
print(f"{'='*60}")
print(f"🤖 MODEL      : {MODEL_ID}")
print(f"📛 SERVED AS  : {SERVED_MODEL_NAME}")
print(f"🔗 BASE URL   : {public_url}/v1")
print(f"{'='*60}\n")

# Lệnh chạy Server - Tối ưu cho Kaggle T4x2
command = [
    "python3", "-m", "vllm.entrypoints.openai.api_server",
    "--model", MODEL_ID,
    "--served-model-name", SERVED_MODEL_NAME,
    "--dtype", "float16",             # float16 cho T4
    "--gpu-memory-utilization", "0.9",
    "--tensor-parallel-size", "2",    # 2 GPU T4
    "--enforce-eager",                # Tắt CUDA graphs, tiết kiệm VRAM
    "--max-model-len", "16384",
    "--host", "0.0.0.0",
    "--port", "8000",
    "--disable-log-requests",
    "--disable-log-stats",
]

def run_server():
    env = os.environ.copy()
    
    # ✅ QUAN TRỌNG: Dùng TORCH_SDPA cho T4 (compute capability 7.5)
    # Flash Attention 2 yêu cầu compute capability >= 8.0
    # XFORMERS đã bị loại bỏ trong vLLM 0.12.0
    env['VLLM_ATTENTION_BACKEND'] = 'TORCH_SDPA'
    
    # ✅ Tắt V1 engine để tránh lỗi trên T4
    env['VLLM_USE_V1'] = '0'
    
    print("⏳ Đang khởi động Server (TORCH_SDPA backend cho T4)...\n")
    
    process = subprocess.Popen(
        command,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
        bufsize=1,
        env=env
    )
    
    # In log real-time
    for line in process.stdout:
        print(line, end="")

# Chạy server
run_server()

In [ ]:
# ==============================================================================
# KEEP-ALIVE LOOP - GIỮ SESSION KAGGLE SỐNG
# ==============================================================================
# Chạy cell này SAU KHI server đã khởi động thành công
# Vòng lặp này giữ cho session Kaggle không bị timeout

import time

print("🔄 BẮT ĐẦU KEEP-ALIVE LOOP")
print("⚠️  ĐỪNG ĐÓNG TAB NÀY! Để giữ session Kaggle sống.\n")

while True:
    print("⚙️ Still running...")
    time.sleep(60)
